<a href="https://colab.research.google.com/github/Satya-roy/Machine_Learning/blob/master/Chat_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

#Libraries needed for Tensorflow processing

import numpy as np

import random
import json

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [31]:

!pip install tensorflow==1.13.2

In [32]:
import tensorflow as tf

In [4]:
!pip install tflearn

     |████████████████████████████████| 102kB 2.3MB/s 
  Created wheel for tflearn: filename=tflearn-0.3.2-cp36-none-any.whl size=128208 sha256=c9953c981b32b61e413970af43414e86af4c5a843d1c395adda92281d0996851
  Stored in directory: /root/.cache/pip/wheels/d0/f6/69/0ef3ee395aac2e5d15d89efd29a9a216f3c27767b43b72c006
Successfully built tflearn


In [33]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

In [34]:
from google.colab import files 
files.upload()

KeyboardInterrupt: ignored

In [5]:
with open('intents.json') as json_data:
  intents = json.load(json_data)

{'intents': [{'context_set': '',
   'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'],
   'responses': ['Hello, thanks for visiting',
    'Good to see you again',
    'Hi there, how can I help?'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks', 'Thank you', "That's helpful"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'tag': 'thanks'},
  {'patterns': ['What hours are you open?',
    'What are your hours?',
    'When are you open?'],
   'responses': ["We're open every day 9am-9pm",
    'Our hours are 9am-9pm every day'],
   'tag': 'hours'},
  {'patterns': ['What is your location?',
    'Where are you located?',
    'What is your address?',
    'Where is your restaurant situated?'],
   'responses': ['We are on the intersection of London Alley and Bridge A

In [6]:
words = []
classes = []
documents = []
ignore = ['?']

#loop through each sentence in the intent's patterns
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # tokenize each and every word in the sentence
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    #add word(s) to documents
    documents.append((w,intent['tag']))
    # add tags to our classes list
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

In [7]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents),"documents",documents)
print(len(classes),"classes",classes)
print(len(words),"unique stemmed words",words)

31 documents [(['Hi'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['What', 'hours', 'are', 'you', 'open', '?'], 'hours'), (['What', 'are', 'your', 'hours', '?'], 'hours'), (['When', 'are', 'you', 'open', '?'], 'hours'), (['What', 'is', 'your', 'location', '?'], 'location'), (['Where', 'are', 'you', 'located', '?'], 'location'), (['What', 'is', 'your', 'address', '?'], 'location'), (['Where', 'is', 'your', 'restaurant', 'situated', '?'], 'location'), (['Do', 'you', 'take', 'credit', 'cards', '?'], 'payments'), (['Do', 'you', 'accept', 'Mastercard', '?'], 'payments'), (['Are', 'you', 'cash', 'only', '?'], 'payments'), (['What', 'is', 'your', 'menu', 'for', 'today', '?'], 'todaysmenu'), (['What', 'are', '

Neural Network



In [10]:
# create trainig data
training = []
output = []

output_empty = [0] * len(classes)

#create training set, bag of words for each sentence

for doc in documents:
  bag = []
  pattern_words = doc[0]
  pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]

  for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(doc[1])] = 1

  training.append([bag, output_row])


random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

In [12]:
tf.reset_default_graph()

net = tflearn.input_data(shape = [None,len(train_x[0])])
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net,10)
net = tflearn.fully_connected(net, len(train_y[0]), activation = 'softmax')
net = tflearn.regression(net)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [14]:
model = tflearn.DNN(net,tensorboard_dir = 'tflearn_logs')
model.fit(train_x,train_y, n_epoch = 1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | time: 0.015s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 24/31
Training Step: 4000  | time: 0.020s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 1.0000 -- iter: 31/31
--
INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [18]:
import pickle 
pickle.dump({'words':words,'classes':classes, 'train_x':train_x,'train_y':train_y},open("training_data","wb"))

In [20]:
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [21]:
with open('intents.json') as json_data:
    intents = json.load(json_data)


In [22]:
model.load('./model.tflearn')


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/model.tflearn


In [35]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [36]:
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [37]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return print(random.choice(i['responses']))

            results.pop(0)

In [38]:
classify('What are you hours of operation?')


[('hours', 0.9667891)]

In [39]:
response('What are you hours of operation?')

Our hours are 9am-9pm every day


In [47]:
response('hello! how are you?')

Hi there, how can I help?
